In [6]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = ""

In [2]:
! pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.3 MB/s eta 0:00:00


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [8]:
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [9]:
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [10]:
parser = JsonOutputParser(pydantic_object=Topic)

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

In [12]:
chain = prompt | model | parser

In [14]:
question = "군대에 대해 자세히 알려줘"
chain.invoke({"question": question})

{'description': '군대는 국가의 방위를 책임지는 조직으로, 육군, 해군, 공군 등으로 구성되어 있으며, 국가의 주권과 영토를 보호하는 역할을 수행합니다.',
 'hashtags': '#군대 #국방 #안보'}

In [15]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser  # 체인을 구성합니다.

chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 인간 활동에 의해 발생하는 온실가스 배출이 주요 원인입니다. 이는 극지방의 빙하가 녹고 해수면이 상승하며, 기후 패턴이 변화하는 등 다양한 환경적 문제를 초래합니다.',
 'hashtags': '#지구온난화 #기후변화 #환경문제'}